## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

import os, sys
currentdir = os.path.dirname(os.path.realpath("Vacation_Search"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Vila Franca Do Campo,PT,37.72,-25.43,57.20,76,20,20.80,few clouds
1,1,Jamestown,US,42.10,-79.24,16.00,85,90,17.20,light snow
2,2,Carnarvon,AU,-24.87,113.63,84.20,66,0,21.92,clear sky
3,3,Fairbanks,US,64.84,-147.72,-0.40,77,75,2.89,broken clouds
4,4,Rikitea,PF,-23.12,-134.97,77.61,79,100,15.17,light rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 32
What is the maximum temperature you would like for your trip? 55


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
12,12,Bluff,NZ,-46.60,168.33,46.99,80,32,1.99,scattered clouds
14,14,Shache,CN,38.42,77.24,33.49,72,32,0.94,scattered clouds
15,15,Tuatapere,NZ,-46.13,167.68,46.99,80,75,1.99,broken clouds
17,17,Brownsville,US,25.90,-97.50,46.40,75,1,5.28,clear sky
34,34,Springville,US,40.17,-111.61,32.00,85,1,4.70,clear sky
47,47,Mataura,NZ,-46.19,168.86,46.99,80,42,1.99,scattered clouds
51,51,Port Hardy,CA,50.70,-127.42,44.60,87,90,12.75,light rain
53,53,Riviere-Au-Renard,CA,49.00,-64.40,48.20,93,90,6.93,overcast clouds
55,55,Culiacancito,MX,24.82,-107.53,52.00,95,0,1.43,clear sky
59,59,Kaitangata,NZ,-46.28,169.85,48.99,67,0,4.00,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID        146
City           146
Country        146
Lat            146
Lng            146
Max Temp       146
Humidity       146
Cloudiness     146
Wind Speed     146
Description    146
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
12,Bluff,NZ,46.99,scattered clouds,-46.60,168.33,
14,Shache,CN,33.49,scattered clouds,38.42,77.24,
15,Tuatapere,NZ,46.99,broken clouds,-46.13,167.68,
17,Brownsville,US,46.40,clear sky,25.90,-97.50,
34,Springville,US,32.00,clear sky,40.17,-111.61,
47,Mataura,NZ,46.99,scattered clouds,-46.19,168.86,
51,Port Hardy,CA,44.60,light rain,50.70,-127.42,
53,Riviere-Au-Renard,CA,48.20,overcast clouds,49.00,-64.40,
55,Culiacancito,MX,52.00,clear sky,24.82,-107.53,
59,Kaitangata,NZ,48.99,clear sky,-46.28,169.85,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()     
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = WeatherPy_vacation.csv
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
